Tài liệu này mang giấy phép Creative Commons Attribution (CC BY). (c) Nguyễn Ngọc Sáng, Zhukovsky 06/2019.

[@SangVn](https://github.com/SangVn) [@VnCFD](https://vncfdgroup.wordpress.com/)

*Thực hành CFD với Python!*

# Bài 22. Phương pháp tính dòng qua mặt, xử lý kết quả, module solver

## 1. Phương pháp tính dòng qua mặt 
Quay lại với bài toán Riemann hay còn gọi là bài toàn phân rã gián đoạn đã được giới thiệu ở phần 2 với phương pháp giải Godunov:
$$U_t + F(U)_x = 0 \qquad(1)\\
U(t=0)= 
\begin{cases}
U_L; \quad x<x_0 \\ 
U_R; \quad x>x_0
\end{cases}
$$

Sử dụng công thức xấp xỉ: 
$$\frac{U_i^{n+1} - U_i^n}{\Delta t} + \frac{F_{i+1/2} - F_{i-1/2}}{\Delta x} = 0 \qquad(2)$$
Theo phương pháp Godunov dòng qua mặt $F_{i+1/2}=F(U_{i+1/2})$. Ngoài ra, có thể tính dòng F  bằng rất nhiều phương pháp khác như HLL, Roe, Osher... Hãy đọc quyển sách của `Eleuterio F. Toro Riemann Solvers and Numerical Methods for Fluid Dynamics`. Trong khuôn khổ phần 3 ta sẽ sử dụng thêm phương pháp Roe. Về code, nếu bạn biết ngôn ngữ FORTRAN có thể tìm hiểu thêm trên trang [I do like CFD](http://www.cfdbooks.com/) của `Katate Masatsuka`. Phần python code dưới đây được chuyển đổi từ fortran code của tác giả trên.

In [1]:
# coding: utf-8
# Module fluxes.py
# Nguyên mẫu FORTRAN - Katate Masatsuka

import numpy as np

def flux_roe(side, PL, PR):
    gamma = 1.4
    nx = side.normal[0]
    ny = side.normal[1]
    tx = -ny
    ty = nx

    # Left state
    rhoL, vxL, vyL, pL = PL[0], PL[1], PL[2], PL[3]
    vnL = vxL * nx + vyL * ny
    vtL = vxL * tx + vyL * ty
    aL = (gamma * pL / rhoL) ** 0.5
    HL = pL / rhoL * gamma / (gamma - 1) + 0.5 * (vxL ** 2 + vyL ** 2)

    # Right state
    rhoR, vxR, vyR, pR = PR[0], PR[1], PR[2], PR[3]
    vnR = vxR * nx + vyR * ny
    vtR = vxR * tx + vyR * ty
    aR = (gamma * pR / rhoR) ** 0.5
    HR = pR / rhoR * gamma / (gamma - 1) + 0.5 * (vxR ** 2 + vyR ** 2)

    # First compute the Roe Averages
    RT = (rhoR / rhoL) ** 0.5
    rho = RT * rhoL
    vx = (vxL + RT * vxR) / (1.0 + RT)
    vy = (vyL + RT * vyR) / (1.0 + RT)
    H = (HL + RT * HR) / (1.0 + RT)
    a = ((gamma - 1.0) * (H - 0.5 * (vx * vx + vy * vy))) ** 0.5
    vn = vx * nx + vy * ny
    vt = vx * tx + vy * ty

    # Wave Strengths
    drho = rhoR - rhoL
    dp = pR - pL
    dvn = vnR - vnL
    dvt = vtR - vtL

    dV = [0.0, 0.0, 0.0, 0.0]
    dV[0] = (dp - rho * a * dvn) / (2.0 * a * a)
    dV[1] = rho * dvt / a
    dV[2] = drho - dp / (a * a)
    dV[3] = (dp + rho * a * dvn) / (2.0 * a * a)

    # Wave Speed
    ws = [0.0, 0.0, 0.0, 0.0]
    ws[0] = abs(vn - a)
    ws[1] = abs(vn)
    ws[2] = abs(vn)
    ws[3] = abs(vn + a)

    # Harten's Entropy Fix JCP(1983), 49, pp357-393:
    # only for the nonlinear fields.
    dws = [0.2, 0.0, 0.0, 0.2]
    if (ws[0] < dws[0]): ws[0] = 0.5 * (ws[0] * ws[0] / dws[0] + dws[0])
    if (ws[3] < dws[3]): ws[3] = 0.5 * (ws[3] * ws[3] / dws[3] + dws[3])

    # Right Eigenvectors
    Rv = np.zeros((4, 4))
    Rv[0, 0] = 1.0
    Rv[1, 0] = vx - a * nx
    Rv[2, 0] = vy - a * ny
    Rv[3, 0] = H - vn * a

    Rv[0, 1] = 0.0
    Rv[1, 1] = a * tx
    Rv[2, 1] = a * ty
    Rv[3, 1] = vt * a

    Rv[0, 2] = 1.0
    Rv[1, 2] = vx
    Rv[2, 2] = vy
    Rv[3, 1] = 0.5 * (vx * vx + vy * vy)

    Rv[0, 3] = 1.0
    Rv[1, 3] = vx + a * nx
    Rv[2, 3] = vy + a * ny
    Rv[3, 3] = H + vn * a

    # Dissipation Term
    diss = np.zeros(4)
    for i in range(4):
        for j in range(4):
            diss[i] += ws[j] * dV[j] * Rv[i, j]

    # Compute the flux.
    fL = np.zeros(4)
    fL[0] = rhoL * vnL
    fL[1] = fL[0] * vxL + pL * nx
    fL[2] = fL[0] * vyL + pL * ny
    fL[3] = fL[0] * HL

    fR = np.zeros(4)
    fR[0] = rhoR * vnR
    fR[1] = fR[0] * vxR + pR * nx
    fR[2] = fR[0] * vyR + pR * ny
    fR[3] = fR[0] * HR

    Roe = 0.5 * (fL + fR - diss) * side.area
    return Roe

Hãy lưu phần code trên vào file `fluxes.py` cũng như hãy tìm hiểu và thêm vào những phương pháp khác tính flux.

## 2. Xử lý kết quả 

Chúng ta cần các hàm để xử lý kết quả hay trường thông số dòng chảy ban đầu. Các hàm cần thiết gồm: ghi, đọc và biểu diễn. Các hàm này sẽ được lưu trong module `functions.py`. Thông số dòng chảy có thể lưu ở ba dạng (tìm hiểu: `Tecplot Data Format Guide`): 
- lưu tại tâm ô lưới: tọa độ tâm ô lưới, P tại tâm (cell_data)
- lưu theo kiểu block: tạo độ điểm lưới, P tại tâm (block_data)
- lưu tại điểm lưới: tọa độ điểm lưới, P tại điểm lưới (point_data)

Trong cấu trúc dữ liệu Cell giá trị P được xác tại tâm ô lưới, ta dễ dàng viết hàm lưu lưới theo cách một và hai. Tuy nhiên để lưu dữ liệu theo cách ba cần có hàm xác định P tại các điểm lưới. Việc này khá phức tạp nên ta sẽ dùng ParaView để chuyển từ dạng hai về dạng ba (dùng filter: `Cell Data To Point Data`) thay vì tự viết code. 

**Thêm vào module functions.py:**

In [3]:
# hàm tính số mach
def Mach(P):
    a = (gamma*P[3]/P[0]) ** 0.5
    u = (P[1]*P[1] + P[2]*P[2]) ** 0.5
    M = u/a
    return M

R_gas = 287.052873836 # Hàng số chất khí
# hàm xác định khối lượng riêng phụ thuộc nhiệt độ và áp suất theo phương trình trạng thái
def rho(T, p):
    rho = p/(R_gas*T)
    return rho

def Temperature(rho, p):
    T = p/(R_gas*rho)
    return T

# Lưu tọa độ tâm ô lưới và thông số dòng chảy tại đó
# Two-Dimensional Field Plots
def write_cell_data(cells, iter, time, filename):
    print('\nWrite cell data to: %s\n' % filename)
    f = open(filename, 'w')
    f.write('TITLE = "vncfd field: iter= %d, time= %f"\n' % (iter, time))
    f.write('VARIABLES = "X", "Y", "rho", "u", "v", "p", "Mach", "T"\n')
    f.write('ZONE T="1", I=%d, J=%d, DATAPACKING=POINT\n' % (cells.size[1], cells.size[0]))
    for cell in cells:
        C = cell.center
        P = cell.P
        M = Mach(P) # hàm xác định số mach
        T = Temperature(P[0], P[3])
        f.write('%f %f %f %f %f %f %f %f\n' % (C[0], C[1], P[0], P[1], P[2], P[3], M, T))
    f.close()

# Lưu tạo độ điểm lưới và thông số dòng chảy tại tâm ô lưới
# Cell-Centered Data
def write_block_data(cells, points, iter, time, filename):
    print('\nWrite block data to: %s\n' % filename)
    f = open(filename, 'w')
    f.write('TITLE = "vncfd field: iter= %d, time= %f"\n' % (iter, time))
    f.write('VARIABLES = "X", "Y", "rho", "u", "v", "p", "Mach", "T"\n')
    f.write('ZONE T="1", I=%d, J=%d, DATAPACKING=BLOCK, VARLOCATION=([3,4,5,6,7,8]=CELLCENTERED)\n' % (points.shape[1], points.shape[0]))

    X_p, Y_p = points[:, :, 0].ravel(), points[:, :, 1].ravel()
    for x in X_p: f.write('%f ' % x)
    f.write('\n')
    for y in Y_p: f.write('%f ' % y)
    f.write('\n')

    for i in range(4):
        for cell in cells: f.write('%f ' % cell.P[i])
        f.write('\n')
    for cell in cells:
        M = Mach(cell.P)
        f.write('%f ' % M)
    for cell in cells:
        T = Temperature(cell.P[0], cell.P[3])
        f.write('%f ' % T)
    f.write('\n')
    f.close()

# đọc các thông số ban đầu (rho, u, v, p) từ file cell_data 
def read_field(cells, filename):
    print('\nRead cell data from: %s\n' % filename)

    f = open(filename, 'r')
    line = f.readline()   # đọc dòng đầu tiên chứa iter và time
    words = line.split()  # chia dòng cuối ra thành các từ riêng biệt bằng dấu cách ' '
    time = float(words[-1].replace('"', '')) # từ cuối cùng bỏ dấu '"'là time
    iter = int(words[-3].replace(',', ''))  # từ thứ 3 tứ cuối lên bỏ dấu ',' là iter
    f.close()

    data = loadtxt(filename, skiprows=3, usecols=(2, 3, 4, 5)) # rho u v p
    for i in range(cells.len):
        cells[i].P = data[i]
        cells[i].U = P2U(data[i]) # hàm tính U từ P
    return iter, time

# Hàm biểu diễn kết quả bằng pyplot: pcolor hay contourf
def show_field(cells, points, porc=0): #porc=0 - pcolor, porc=1 - contourf
    Nj, Ni = cells.size[0], cells.size[1]

    if porc == 0:
        # Nếu vẽ pcolor, pcolormesh... cần tọa độ tại đỉnh, giá trị tại tâm
        X_p, Y_p = points[:, :, 0], points[:, :, 1]
    else:
        # Nếu vẽ contour, contourf, quiver, streamplot... cần tọa độ tại tâm, giá trị tại tâm
        centers = array([cell.center for cell in cells]).reshape((Nj, Ni, 2))
        X_c, Y_c = centers[:, :, 0], centers[:, :, 1]

    fig, axs = plt.subplots(2, 2)
    titles = ['rho', 'u', 'v', 'p']
    i = 0
    for ax in axs.flat:
        value_c = array([cell.P[i] for cell in cells]).reshape((Nj, Ni))
        if porc == 0: c = ax.pcolor(X_p, Y_p, value_c)
        else: c = ax.contourf(X_c, Y_c, value_c)
        ax.set_title(titles[i])
        fig.colorbar(c, ax=ax)
        i += 1

    fig.tight_layout()
    plt.show()

# Chuyển dữ liệu từ dạng block_data sang point_data:
# Bước 1. dùng ParaView mở file block_data
# Bước 2. Apply filter: CellDataToPointData
# Bước 3. xuất dữ liệu tại điểm lưới: save data -> 'filename.txt'
def show_point_data(Nj, Ni, filename):
    data = loadtxt(filename, skiprows=1, delimiter=',') # file thu được từ paraview chuyển dạng 2 sang 3
    # Dòng đầu tiên: "rho", "u", "v", "p", "Mach", "T", "Points:0", "Points:1", "Points:2"
    # tương ứng các cột: 0, 1, 2, 3, 4, 5, 6, 7, 8 

    x = data[:, 6].reshape((Nj, Ni)) # cột thứ 5
    y = data[:, 7].reshape((Nj, Ni)) # cột thứ 6

    fig, axs = plt.subplots(2, 2)
    titles = ['rho', 'u', 'v', 'p', 'Mach', 'T']
    iv = [0, 3, 4, 5]

    i = 0
    for ax in axs.flat:
        value = data[:, iv[i]].reshape((Nj, Ni))

        c = ax.contourf(x, y, value)
        ax.set_title(titles[iv[i]])
        # ax.set_xlim([-0.5, 1.5])
        # ax.set_ylim([-0.5, 0.5])
        fig.colorbar(c, ax=ax)
        i += 1

    fig.tight_layout()
    img_filename = filename.replace('.txt', '.png')
    plt.savefig(img_filename)
    plt.show()

### 3. Module solver

Hàm solver - giải hệ phương trình Euler 2D có cấu trúc tương tự như trong trường hợp 1D ở phần 2. Nó bao gồm các bước:

In [ ]:
# coding: utf-8
# Module solver.py

from functions import write_cell_data
import setup

# Hàm eu_solver thực hiện các bước lặp để tìm nghiệm
# Biến đầu vào gồm có: các ô lưới, các mặt, số vòng lặp và thời gian lúc ban đầu 
def eu_solver(cells, sides, iter, time):
    # thiết lập điều kiện biên
    setup.set_boco(sides)

    while(time < setup.time_target):
        iter += 1 # tăng số vòng lặp lên 1 đơn vị
    
        # tính bước thời gian
        dt = cells.time_step_global(setup.CFL) # có thể thiết lập dt trong setup: dt = setup.dt
        if(time+dt > setup.time_target): dt = setup.time_target - time
        time += dt
        print('iteration: %d, dt: %f, time: %f' % (iter, dt, time))

        # tính dòng qua các bề mặt
        sides.flux_boundaries()          # dòng qua các mặt biên
        sides.flux_inner_sides(setup.flux_func) # dòng qua các mặt trong

        # tính giá trị U mới
        cells.new_U(dt)

        # tính giá trị P mới
        cells.new_P()

        # nếu số vòng lặp bằng iter_to_write_field thì ghi lại kết quả giữa chừng 
        if setup.write_field_iter is not None and not (iter%setup.write_field_iter):
            write_cell_data(cells, iter, time, setup.field_filename)
    
    # ghi lại kết quả cuối cùng
    write_cell_data(cells, iter, time, setup.field_filename)

Ở đây, các biến như: time_target, flux_func, write_field_iter.. được xác định trong module `setup.py`, sẽ được giới thiệu ở bài sau.

Trong nhiều trường hợp ta muốn chạy một số vòng lặp nhất định chứ không cần quan tâm thời gian cuối cùng, do đó ta sửa lại hàm `eu_solver` như sau:

In [ ]:
# Hàm eu_solver thực hiện các bước lặp để tìm nghiệm
# Biến đầu vào gồm có: các ô lưới, các mặt, số vòng lặp, thời gian lúc ban đầu
def eu_solver(cells, sides, iter, time):
    # thiết lập điều kiện biên
    setup.set_boco(sides)

    # tính theo thời gian
    if setup.time_target is not None:
        while(time < setup.time_target):
            iter += 1  # tăng số vòng lặp lên 1 đơn vị
            # tính bước thời gian
            dt = cells.time_step_global(setup.CFL)  # có thể thiết lập dt trong setup: dt = setup.dt
            if (time + dt > setup.time_target): dt = setup.time_target - time
            time += dt
            print('iteration: %d, dt: %f, time: %f' % (iter, dt, time))
            iter, time = iteration(cells, sides, iter, time, dt)

    # tính theo số vòng lặp
    elif setup.iter_target is not None:
        while(iter < setup.iter_target):
            iter += 1  # tăng số vòng lặp lên 1 đơn vị
            # tính bước thời gian
            dt = cells.time_step_global(setup.CFL)  # có thể thiết lập dt trong setup: dt = setup.dt
            time += dt
            print('iteration: %d, dt: %f, time: %f' % (iter, dt, time))
            iter, time = iteration(cells, sides, iter, time, dt)

    # ghi lại kết quả cuối cùng
    write_cell_data(cells, iter, time, setup.field_filename)


def iteration(cells, sides, iter, time, dt):
    # tính dòng qua các bề mặt
    sides.flux_boundaries()  # dòng qua các mặt biên
    sides.flux_inner_sides(setup.flux_func)  # dòng qua các mặt trong

    # tính giá trị U mới
    cells.new_U(dt)

    # tính giá trị P mới
    cells.new_P()

    # nếu số vòng lặp bằng write_field_iter thì ghi lại kết quả
    if setup.write_field_iter is not None and not (iter % setup.write_field_iter):
        write_cell_data(cells, iter, time, setup.field_filename)

    return iter, time

Như vậy là chúng ta đã có đủ các hàm cần thiết. Bước cuối cùng, ta cần thiết lập các thông số: điều kiện ban đầu, điều kiện biên, số CFL, thời điểm kết thúc tính toán... và gọi các hàm cần thiết theo thứ tự hợp lý để giải quyết bài toán. 
Để thực hiện những việc này ta dùng hai module `setup.py` và `run.py` sẽ được giới thiệu ở bài sau.

## [Bài 23. Thiết lập module setup, run. Tính bài toán dòng chảy trên âm qua dốc](Bai_23.ipynb)